# CBPP Federal Jobs Guarantee

This analyzes the [CBPP's 2018 Federal Jobs Guarantee proposal](https://www.cbpp.org/research/full-employment/the-federal-job-guarantee-a-policy-to-achieve-permanent-full-employment#_ftn1) from Mark Paul, William Darity, Jr., and Darrick Hamilton. It is primarily described by this table:

![img](https://imgur.com/5Km1yLO.png)

In addition to the \$32,500 average wage, the base wage is \$24,600. The program is available to people age 18 and over.

Modeling the uptake is nontrivial. I estimate each tax unit's benefit as follows:

1. Assign a JG wage $w$ which draws from a random uniform between \$24,600 and \$40,400.
2. Assign a maximum JG benefit $mb$ equal to $w * n_{18-64}$.
3. Assign an actual JG benefit $b$ equal to $max(\$0, mb - wages)$.
4. Assign % FTE (which includes both % FTE while working and share of the year with the JG job) equal to $b / (b+e00200)$ (`e00200` is "wages, salaries, and tips for filing unit"*).
5. Calculate the total FTE across tax units as the weighted sum of % FTE. This is the total FTE the JG would be expected to hire with 100% participation.
6. Divide the expected 9.7 million (from the table) by this FTE total. Call this $p$, the probability that each tax unit will participate in the JG.
7. Randomly assign each tax unit a participation flag with probability $p$.
8. Add $b$ to the `e00200` of tax units flagged as participating.
9. Multiply itemizable state and local income/sales taxes (`e18400`) by the change in `e00200p` (i.e., assume flat SALT--unrealistically).

\* This could be enhanced by splitting `e00200` between `e00200p` and `e00200s` for the taxpayer and spouse, respectively.

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis*

## Setup

### Imports

In [1]:
import taxcalc as tc
import taxcalc_helpers as tch
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
tc.__version__

'0.20.1'

### Settings

In [3]:
sns.set_style('white')
DPI = 500
mpl.rc('savefig', dpi=DPI)
mpl.rcParams['figure.dpi'] = DPI
mpl.rcParams['figure.figsize'] = 6.4, 4.8  # Default.

In [4]:
mpl.rcParams['font.sans-serif'] = 'Roboto'
mpl.rcParams['font.family'] = 'sans-serif'

# Set title text color to dark gray (https://material.io/color) not black.
TITLE_COLOR = '#212121'
mpl.rcParams['text.color'] = TITLE_COLOR

# Axis titles and tick marks are medium gray.
AXIS_COLOR = '#757575'
mpl.rcParams['axes.labelcolor'] = AXIS_COLOR
mpl.rcParams['xtick.color'] = AXIS_COLOR
mpl.rcParams['ytick.color'] = AXIS_COLOR

# Use Seaborn's default color palette.
# https://stackoverflow.com/q/48958426/1840471 for reproducibility.
sns.set_palette(sns.color_palette())

In [5]:
# Specify number of decimals in tables.
pd.set_option('precision', 2)

## Generate data

### Load data

Generate a set of normal CPS records for 2018 using `Calculator`, then extract the dataframe.

In [6]:
calc = tc.Calculator(records=tc.Records.cps_constructor(), 
                     policy=tc.Policy(), verbose=False)
calc.advance_to_year(2018)

In [7]:
cps_raw_cols = pd.read_csv(
    os.path.join(tc.Records.CUR_PATH, 'cps.csv.gz')).columns

In [8]:
df = calc.dataframe(list(cps_raw_cols))

1) Assign a JG wage $w$ which draws from a random uniform between \$24,600 and \$40,400.

In [9]:
JG_MIN_WAGE = 24600
JG_AVG_WAGE = 32500
jg_max_wage = 39800  # Set to calibrate the average wage to $32,500.
# jg_max_wage = JG_AVG_WAGE + (JG_AVG_WAGE - JG_MIN_WAGE)

In [10]:
np.random.seed(1)
df['jg_w'] = np.random.randint(low=JG_MIN_WAGE, high=jg_max_wage, 
                               size=df.shape[0])

In [11]:
print('Simulated JG wages range from ${:,.0f}'.format(df.jg_w.min()) + 
      ' to ${:,.0f}.'.format(df.jg_w.max()))

Simulated JG wages range from $24,600 to $39,799.


2) Assign a maximum JG benefit $mb$ equal to $w * n_{1864}$.

In [12]:
def n65(df):
    return ((df.age_head >= 65).astype(int) + 
            (df.age_spouse >= 65).astype(int) + 
            df.elderly_dependent)

In [13]:
df['n65'] = n65(df)

In [14]:
df['n1864'] = df.n1820 + df.n21 - df.n65
df['jg_mb'] = df.jg_w * df.n1864

3) Assign an actual JG benefit $b$ equal to $max(\$0, mb - wages)$.

In [15]:
df['jg_b'] = np.maximum(0, df.jg_mb - df.e00200)

4) Assign % FTE (which includes both % FTE while working and share of the year with the JG job) equal to $n1864 * b / (b+e00200)$ (`e00200` is "wages, salaries, and tips for filing unit"*).

In [16]:
df['jg_fte'] = df.n1864 * df.jg_b / (df.jg_b + df.e00200)

In [17]:
print('On average, JG workers are simulated to work ' +
      '{:.0f}% of the year in the JG job.'.format(df.jg_fte.mean() * 100))

On average, JG workers are simulated to work 49% of the year in the JG job.


5) Calculate the total FTE across tax units as the weighted sum of % FTE. This is the total FTE the JG would be expected to hire with 100% participation.

In [18]:
total_potential_jg_fte = (df.jg_fte * df.s006).sum()
print('With 100% participation, the JG would hire ' +
      '{:.1f}M people.'.format(total_potential_jg_fte / 1e6))

With 100% participation, the JG would hire 71.8M people.


6) Divide the expected 9.7 million (from the table) by this FTE total. Call this $p$, the probability that each tax unit will participate in the JG.

In [19]:
JG_FTE = 9700000
jg_p = JG_FTE / total_potential_jg_fte
print('To meet the CBPP target of {:.1f}M FTE, '.format(JG_FTE / 1e6) +
      '{:.1f}%'.format(jg_p * 100) + 
      ' of eligible workers would participate in the JG.')

To meet the CBPP target of 9.7M FTE, 13.5% of eligible workers would participate in the JG.


7) Randomly assign each tax unit a participation flag with probability $p$.

In [20]:
df['jg_participate'] = np.random.rand(df.shape[0]) < jg_p

8) Add $b$ to the `e00200` of tax units flagged as participating, and split the additional amount between `e00200p` and `e00200s` according to the current split.

In [21]:
df['jg'] = df.jg_b * df.jg_participate

In [22]:
df['jg_m'] = df.jg * df.s006 / 1e6
jg_total_m = df.jg_m.sum()
print('Total simulated wages are ${:.1f}B.'.format(jg_total_m / 1e3))

Total simulated wages are $316.2B.


In [23]:
df[df.jg_fte > 0][['e00200', 'jg', 'e00200p', 'jg_fte', 'n1864', 'jg_w', 
                   'jg_mb', 'jg_b']].sort_values('e00200', 
                                                 ascending=False).head(3)

,e00200,jg,e00200p,jg_fte,n1864,jg_w,jg_mb,jg_b
98011,211638.82,7853.18,0.0,0.21,6.0,36582,219492.0,7853.18
98008,211638.82,0.00,0.0,0.38,6.0,37691,226146.0,14507.18
98001,211638.82,0.00,0.0,0.65,6.0,39562,237372.0,25733.18


In [24]:
avg_jg_wage = ((df.jg * df.s006).sum() / 
               (df.jg_participate * df.jg_fte * df.s006).sum())
print('The average JG wage per FTE is ${:,.0f}'.format(avg_jg_wage) +
      ' (should be close to ${:,.0f} target).'.format(JG_AVG_WAGE))

The average JG wage per FTE is $32,575 (should be close to $32,500 target).


Split JG between primary and spouse.

In [25]:
df['e00200_orig'] = df.e00200
df['e00200p_orig'] = df.e00200p
df['e00200s_orig'] = df.e00200s

In [26]:
df['jgp'] = df.jg * np.where(df.e00200_orig > 0, 
                             df.e00200p_orig / df.e00200_orig, 1)
df['jgs'] = df.jg * np.where(df.e00200_orig > 0, 
                             df.e00200s_orig / df.e00200_orig, 0)

In [27]:
df['e00200'] = df.e00200_orig + df.jg
df['e00200p'] = df.e00200p_orig + df.jgp
df['e00200s'] = df.e00200s_orig + df.jgs

9) Multiply itemizable state and local income/sales taxes (`e18400`) by the change in `e00200p` (i.e., assume flat SALT--unrealistically).

**TODO**

### Clean data

Fix `e00900` pending https://github.com/open-source-economics/Tax-Calculator/issues/2024.

In [28]:
df['e00900'] = df.e00900p + df.e00900s

## Calculate

Run Tax-Calculator.

Include non-health benefits in calculating after-tax income.

In [29]:
reform_no_medicaid_medicare = {
    2018: {
        "_BEN_mcaid_repeal": [True],
        "_BEN_mcare_repeal": [True]
    }
}

In [30]:
recs = tc.Records(data=df,
                  start_year=2018, 
                  weights=tc.Records.CPS_WEIGHTS_FILENAME,
                  adjust_ratios=tc.Records.CPS_RATIOS_FILENAME,
                  benefits=tc.Records.CPS_BENEFITS_FILENAME)

TODO: Provide JG to all records in `jg`, then split records with according weight. This will give more stable results.

In [31]:
jg = tch.calc_df(records=recs,
                 year=2018,
                 reform=reform_no_medicaid_medicare,
                 group_vars=['expanded_income', 'e00200', 'c00100'],
                 metric_vars=['aftertax_income', 'XTOT', 'nu18', 'eitc',
                              'e00200', 'c07220', 'c11070'])

In [32]:
base = tch.calc_df(records=tc.Records.cps_constructor(),
                   year=2018,
                   reform=reform_no_medicaid_medicare,
                   group_vars=['expanded_income', 'e00200', 'c00100'],
                   metric_vars=['aftertax_income', 'XTOT', 'nu18', 'eitc',
                                'e00200', 'c07220', 'c11070'])

Add refundable and nonrefundable CTC components.

In [33]:
jg['ctc_m'] = jg.c07220_m + jg.c11070_m
base['ctc_m'] = base.c07220_m + base.c11070_m

Calculate differences across income measures, EITC, and CTC.

In [34]:
def jg_diff(var):
    diff = jg[var + '_m'].sum() - base[var + '_m'].sum()
    pct_diff = diff / base[var + '_m'].sum()
    print('JG changes ' + var + ' by ${:.1f}B'.format(diff / 1e3) +
          ' ({:.1f}%).'.format(pct_diff * 100))

In [35]:
jg_diff('e00200')
jg_diff('aftertax_income')
jg_diff('eitc')
jg_diff('ctc')

JG changes e00200 by $316.2B (3.9%).
JG changes aftertax_income by $256.1B (2.5%).
JG changes eitc by $-4.4B (-7.3%).
JG changes ctc by $2.6B (3.0%).


### Total cost

Change in after-tax income, plus \$11,000 per FTE in supplies and capital goods. 

Ignore employer's share of FICA (essentially tax revenue) and \$10,000 per job in benefits (until analysis of lost safety net benefits like Medicaid is conducted, assume they roughly balance out).

In [36]:
afti_chg_b = (jg.aftertax_income_m.sum() - base.aftertax_income_m.sum()) / 1e3
SUPPLIES_PER_FTE = 11000
supplies_b = (SUPPLIES_PER_FTE * JG_FTE) / 1e9
total_cost_b = afti_chg_b + supplies_b
print('The total cost of JG is ${:.1f}B'.format(total_cost_b) +
      ': ${:.1f}B in after-tax wages plus '.format(afti_chg_b) +
      '${:.1f}B in supplies and capital goods.'.format(supplies_b))

The total cost of JG is $362.8B: $256.1B in after-tax wages plus $106.7B in supplies and capital goods.


## Analysis

### Who benefits?

In [37]:
jg_share_no_wages = df[(df.e00200_orig == 0)].jg_m.sum() / jg_total_m
print('{:.1f}%'.format(jg_share_no_wages * 100) +
      ' of JG wages go to tax units with no current wages.')

40.7% of JG wages go to tax units with no current wages.


In [38]:
jg_share_snap = df[(df.snap_ben > 0)].jg_m.sum() / jg_total_m
print('{:.1f}%'.format(jg_share_snap * 100) +
      ' of JG wages go to tax units with current SNAP benefits.')

28.6% of JG wages go to tax units with current SNAP benefits.


In [39]:
jg_share_tanf = df[(df.tanf_ben > 0)].jg_m.sum() / jg_total_m
print('{:.1f}%'.format(jg_share_tanf * 100) +
      ' of JG wages go to tax units with current TANF benefits.')

3.5% of JG wages go to tax units with current TANF benefits.


People in tax units benefiting from JG.

In [40]:
jg_tu_XTOT = ((df.jg > 0) * df.s006 * df.XTOT).sum()
jg_tu_n1864 = ((df.jg > 0) * df.s006 * df.n1864).sum()
print('{:.1f}M'.format(jg_tu_XTOT / 1e6) + 
      ' people are in tax units that would benefit from JG, of whom ' +
      '{:.1f}M'.format(jg_tu_n1864 / 1e6) + ' are age 18-64.')

21.8M people are in tax units that would benefit from JG, of whom 15.5M are age 18-64.


### Poverty impact

In [41]:
def add_poverty(df):
    EXTREME_POVERTY_LINE = 780
    df['pov_extreme_m'] = df.XTOT_m * (
        df.aftertax_income < (EXTREME_POVERTY_LINE * df.XTOT))
    df['pov_extreme_child_m'] = df.nu18_m * (
        df.aftertax_income < (EXTREME_POVERTY_LINE * df.XTOT))
    df['pov_10k_m'] = df.XTOT_m * (
        df.aftertax_income < (10000 * df.XTOT))
    df['pov_10k_child_m'] = df.nu18_m * (
        df.aftertax_income < (10000 * df.XTOT))
    # Use $7,500 threshold as that's what NIT could guarantee.
    df['pov_7500_m'] = df.XTOT_m * (
        df.aftertax_income < (7500 * df.XTOT))
    df['pov_7500_child_m'] = df.nu18_m * (
        df.aftertax_income < (7500 * df.XTOT))
    df['fpl_m'] = df.XTOT_m * (df.c00100 < tch.fpl(df.XTOT))
    df['fpl_child_m'] = df.nu18_m * (df.c00100 < tch.fpl(df.XTOT))
    df['fpla_m'] = df.XTOT_m * (df.aftertax_income < tch.fpl(df.XTOT))
    df['fpla_child_m'] = df.nu18_m * (df.aftertax_income < tch.fpl(df.XTOT))

In [42]:
add_poverty(jg)
add_poverty(base)

In [43]:
def print_poverty(numerator, denominator='XTOT_m'):
    jg_rate = jg[numerator].sum() / jg[denominator].sum()
    base_rate = base[numerator].sum() / base[denominator].sum()
    chg = 1 - jg_rate / base_rate
    diff = base[numerator].sum() - jg[numerator].sum()
    cost_pp = 1000 * total_cost_b / diff
    print('JG reduces ' + numerator + ' by {:.1f}%'.format(chg * 100) +
          ', from {:,.1f}%'.format(base_rate * 100) + 
          ' to {:,.1f}%. '.format(jg_rate * 100) +
          'It does so at a cost of ${:,.0f} '.format(cost_pp) +
          'per person lifted out of poverty.')

TODO: Put this in a DataFrame.

In [44]:
for i in ['fpla', 'fpl', 'pov_extreme', 'pov_10k', 'pov_7500']:
    print_poverty(i + '_m')
    print_poverty(i + '_child_m', 'nu18_m')

JG reduces fpla_m by 12.0%, from 11.2% to 9.8%. It does so at a cost of $80,574 per person lifted out of poverty.
JG reduces fpla_child_m by 10.7%, from 13.0% to 11.6%. It does so at a cost of $317,006 per person lifted out of poverty.
JG reduces fpl_m by 10.0%, from 26.9% to 24.2%. It does so at a cost of $40,203 per person lifted out of poverty.
JG reduces fpl_child_m by 11.3%, from 28.2% to 25.0%. It does so at a cost of $139,323 per person lifted out of poverty.
JG reduces pov_extreme_m by 12.2%, from 1.6% to 1.4%. It does so at a cost of $555,625 per person lifted out of poverty.
JG reduces pov_extreme_child_m by 10.9%, from 2.0% to 1.8%. It does so at a cost of $1,995,090 per person lifted out of poverty.
JG reduces pov_10k_m by 11.5%, from 18.4% to 16.3%. It does so at a cost of $51,225 per person lifted out of poverty.
JG reduces pov_10k_child_m by 10.2%, from 29.2% to 26.3%. It does so at a cost of $148,699 per person lifted out of poverty.
JG reduces pov_7500_m by 12.4%, from

#### Cost per person lifted out of poverty

In [45]:
pov_7500_m_diff = base.pov_7500_m.sum() - jg.pov_7500_m.sum()
# 1000 * total_cost_b / pov_7500_m_diff

In [46]:
(1e9 * total_cost_b) / (1e6 * pov_7500_m_diff)

78374.51176481046

Comparison to NIT.

In [47]:
78375 / 9243

8.479389808503733